In [0]:
# BATCH PROCESSING: SPARK ON DATABRICKS - PART 2

In [0]:
# Reading the saved parquet files from 'data_cleaning' notebook into dataframes
df_pin = spark.read.parquet("/mnt/0a3db223d459_storage/df_pin")
df_geo = spark.read.parquet("/mnt/0a3db223d459_storage/df_geo")
df_user = spark.read.parquet("/mnt/0a3db223d459_storage/df_user")

In [0]:
# Creating temporary views to run SQL queries
df_pin.createOrReplaceTempView("df_pin_temp_view")
df_geo.createOrReplaceTempView("df_geo_temp_view")
df_user.createOrReplaceTempView("df_user_temp_view")

In [0]:
# Task 4: Find the most popular category in each country

In [0]:
# Find the most popular Pinterest category people post to based on their country.
popular_category_by_country_query = """
                                    WITH CountCategory AS (
                                        SELECT
                                            geo.country,
                                            pin.category,
                                            COUNT(*) as category_count
                                        FROM df_geo_temp_view AS geo
                                        JOIN df_pin_temp_view AS pin 
                                            ON geo.ind = pin.ind
                                        GROUP BY 
                                            geo.country, pin.category
                                    ),
                                    MaxCategoryCount AS (
                                        SELECT
                                            country,
                                            MAX(category_count) as max_category_count
                                        FROM CountCategory
                                        GROUP BY country
                                    )

                                    SELECT
                                        cc.country,
                                        cc.category,
                                        cc.category_count
                                    FROM CountCategory AS cc
                                    JOIN MaxCategoryCount AS mcc
                                        ON cc.country = mcc.country
                                        AND cc.category_count = mcc.max_category_count
                                    ORDER BY 
                                        country, category_count DESC;
                                    """
display(spark.sql(popular_category_by_country_query).head(10))

country,category,category_count
Afghanistan,education,2
Albania,mens-fashion,16
Algeria,quotes,4
American Samoa,beauty,2
American Samoa,tattoos,2
Andorra,quotes,2
Angola,diy-and-crafts,5
Anguilla,home-decor,4
Antarctica (the territory South of 60 deg S),christmas,4
Antigua and Barbuda,art,3


In [0]:
# Task 5: Find which was the most popular category each year

In [0]:
# Find how many posts each category had between 2018 and 2022.
popular_category_annually_query =   """
                                    SELECT
                                        YEAR(geo.timestamp) AS post_year,
                                        pin.category,
                                        COUNT(*) AS category_count
                                    FROM df_geo_temp_view AS geo
                                    JOIN df_pin_temp_view AS pin
                                        ON geo.ind = pin.ind
                                    WHERE
                                        YEAR(geo.timestamp) BETWEEN 2018 AND 2022
                                    GROUP BY
                                        YEAR(geo.timestamp), pin.category
                                    ORDER BY
                                        post_year DESC, category_count DESC;
                                    """
display(spark.sql(popular_category_annually_query).head(10))

post_year,category,category_count
2022,beauty,8
2022,christmas,8
2022,quotes,6
2022,diy-and-crafts,4
2022,vehicles,4
2022,mens-fashion,3
2022,art,3
2022,tattoos,3
2022,education,2
2022,home-decor,2


In [0]:
# Task 6: Find the user with most followers in each country

In [0]:
# Part 1: For each country find the user with the most followers.
top_user_by_country_query = """
                            WITH RankedUsers AS (
                                SELECT DISTINCT
                                    g.country,
                                    p.poster_name,
                                    p.follower_count,
                                    RANK() OVER (PARTITION BY g.country ORDER BY p.follower_count DESC) as rank
                                FROM df_geo_temp_view g
                                JOIN df_pin_temp_view p ON g.ind = p.ind
                            )

                            SELECT
                                country,
                                poster_name,
                                follower_count
                            FROM RankedUsers
                            WHERE rank = 1
                            ORDER BY country, poster_name
                            """
display(spark.sql(top_user_by_country_query).head(10))

# Part 2: Based on the above query, find the country with the user with most followers.
country_with_top_user_query= f"""
                            SELECT
                                country,
                                follower_count
                            FROM ({top_user_by_country_query})
                            ORDER BY 
                                follower_count DESC
                            LIMIT 1;
                            """
display(spark.sql(country_with_top_user_query).head(10))

country,poster_name,follower_count
Afghanistan,9GAG,3000000
Albania,The Minds Journal,5000000
Algeria,YourTango,942000
American Samoa,Mamas Uncut,8000000
Andorra,Teachers Pay Teachers,1000000
Angola,Tastemade,8000000
Anguilla,"Kristen | Lifestyle, Mom Tips & Teacher Stuff Blog",92000
Antarctica (the territory South of 60 deg S),Refinery29,1000000
Antigua and Barbuda,Country Living Magazine,1000000
Argentina,Next Luxury,800000


country,follower_count
American Samoa,8000000


In [0]:
# Task 7: Find the most popular category for different age groups

In [0]:
popular_category_by_age_group_query =   """
                                        WITH AgeGroupCategories AS (
                                            SELECT
                                                CASE
                                                    WHEN users.age >= 18 AND users.age <= 24 THEN '18-24'
                                                    WHEN users.age >= 25 AND users.age <= 35 THEN '25-35'
                                                    WHEN users.age >= 36 AND users.age <= 50 THEN '36-50'
                                                    WHEN users.age > 50 THEN '50+'
                                                END AS age_group,
                                                pin.category
                                            FROM df_user_temp_view AS users
                                            JOIN df_pin_temp_view AS pin 
                                                ON users.ind = pin.ind
                                        ),
                                        CategoryCounts AS (
                                            SELECT
                                                age_group,
                                                category,
                                                COUNT(*) AS category_count
                                            FROM AgeGroupCategories
                                            GROUP BY age_group, category
                                        ),
                                        RankedCategories AS (
                                            SELECT
                                                age_group,
                                                category,
                                                category_count,
                                                RANK() OVER (PARTITION BY age_group ORDER BY category_count DESC) as category_rank
                                            FROM CategoryCounts
                                        )
                                        SELECT
                                            age_group,
                                            category,
                                            category_count
                                        FROM RankedCategories
                                        WHERE category_rank = 1
                                        ORDER BY age_group;
                                        """
display(spark.sql(popular_category_by_age_group_query).head(10))

age_group,category,category_count
18-24,mens-fashion,31
25-35,diy-and-crafts,23
36-50,finance,17
50+,beauty,3
50+,education,3


In [0]:
# Task 8: Find the median follower count for different age groups

In [0]:
median_follower_count_by_age_group_query =  """
                                            WITH AgeGroupCategories AS (
                                                SELECT
                                                    CASE
                                                        WHEN users.age >= 18 AND users.age <= 24 THEN '18-24'
                                                        WHEN users.age >= 25 AND users.age <= 35 THEN '25-35'
                                                        WHEN users.age >= 36 AND users.age <= 50 THEN '36-50'
                                                        WHEN users.age > 50 THEN '50+'
                                                    END AS age_group,
                                                    pin.follower_count
                                                FROM df_user_temp_view AS users
                                                JOIN df_pin_temp_view AS pin 
                                                    ON users.ind = pin.ind
                                            )
                                            SELECT
                                                age_group,
                                                percentile_approx(follower_count, 0.5) AS median_follower_count
                                            FROM AgeGroupCategories
                                            GROUP BY age_group
                                            ORDER BY age_group;
                                            """
display(spark.sql(median_follower_count_by_age_group_query))

age_group,median_follower_count
18-24,171000
25-35,46000
36-50,3000
50+,3000


In [0]:
# Task 9: Find how many users have joined each year?

In [0]:
# Find how many users have joined between 2015 and 2020.
annual_user_join_count_query="""
                            SELECT
                                YEAR(date_joined) AS post_year,
                                COUNT(*) AS number_users_joined
                            FROM df_user_temp_view
                            WHERE 
                                YEAR(date_joined) BETWEEN 2015 AND 2020
                            GROUP BY
                                YEAR(date_joined)
                            ORDER BY
                                post_year DESC;
                            """
display(spark.sql(annual_user_join_count_query))

post_year,number_users_joined
2017,41
2016,116
2015,100


In [0]:
# Task 10: Find the median follwoer count of users based on thei joining year

In [0]:
# Find the median follower count of users have joined between 2015 and 2020.
median_follower_count_by_join_year_query =  """
                                            SELECT
                                                YEAR(users.date_joined) AS post_year,
                                                percentile_approx(pin.follower_count, 0.5) AS median_follower_count
                                            FROM df_user_temp_view AS users
                                            JOIN df_pin_temp_view AS pin
                                                ON users.ind = pin.ind
                                            WHERE
                                                YEAR(users.date_joined) BETWEEN 2015 AND 2020
                                            GROUP BY
                                                YEAR(users.date_joined)
                                            """
display(spark.sql(median_follower_count_by_join_year_query))

post_year,median_follower_count
2015,51000
2016,41000
2017,6000


In [0]:
# Task 11: Find the median follower count of users based on their joining year and age group

In [0]:
# Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.
median_follower_age_join_year_query =   """
                                        SELECT
                                            CASE
                                                WHEN users.age >= 18 AND users.age <= 24 THEN '18-24'
                                                WHEN users.age >= 25 AND users.age <= 35 THEN '25-35'
                                                WHEN users.age >= 36 AND users.age <= 50 THEN '36-50'
                                                WHEN users.age > 50 THEN '50+'
                                            END AS age_group,
                                            YEAR(users.date_joined) AS post_year,
                                            percentile_approx(pin.follower_count, 0.5) AS median_follower_count
                                        FROM df_user_temp_view AS users
                                        JOIN df_pin_temp_view AS pin 
                                            ON users.ind = pin.ind
                                        WHERE
                                            YEAR(users.date_joined) BETWEEN 2015 AND 2020
                                        GROUP BY
                                            age_group, post_year
                                        ORDER BY
                                            age_group, post_year DESC;
                                        """
display(spark.sql(median_follower_age_join_year_query).head(10))

age_group,post_year,median_follower_count
18-24,2017,10000
18-24,2016,60000
18-24,2015,211000
25-35,2017,8000
25-35,2016,43000
25-35,2015,51000
36-50,2017,3000
36-50,2016,11000
36-50,2015,0
50+,2017,5000
